In [2]:
import pandas as pd
import collections

In [ ]:
def loadInvestments(filename):
    df = pd.read_csv(filename)
    
    df2 = pd.concat([df.iloc[:,1:2],df.iloc[:,-13:]],axis = 1)
    df3 = pd.concat([df2.iloc[:,0:2],df2.iloc[:,-1:]],axis = 1)
    
    df3['Return'] = df3['2020-01'] - df3['2019-01']
    df4 = df3[['RegionName', '2020-01', 'Return']]
    
    df4 = df4.rename(columns = {'2020-01': 'Cost'})
    df4 = df4[df4['RegionName'] != 'United States']
    
    return df4

def optimizeInvestments(df, money):
    val = list(df['Return'])

    cost = list(df['Cost'])

        
    n = len(val)
    
    Inv = [[0 for x in range(money + 1)] for x in range(n + 1)]

    for i in range(n + 1):
        for w in range(money + 1):
            if i == 0 or w == 0:
                 Inv[i][w] = 0
            elif cost[i-1] <= w:
                 Inv[i][w] = max(val[i-1]
                          +  Inv[i-1][w-cost[i-1]],  
                               Inv[i-1][w])
            else:
                 Inv[i][w] =  Inv[i-1][w]
                    
    res =  Inv[n][money] 

    w = money
    investmentCosts = []

    for i in range(n, 0, -1): 

        if res <= 0: 
            break

        if res ==  Inv[i - 1][w]: 
            continue
        else: 
            investmentCosts.append(cost[i - 1])

            res = res - val[i - 1] 
            w = w - cost[i - 1] 
  
    return  Inv[n][money], investmentCosts


def getinvests(df, investments, increment):
    investmentsarray = []
    cost = df['Cost']
    val = df['Return']

    for i, row in df.iterrows():
        for x in range(0, len(investments)):
            if df.loc[i, 'Cost'] == investments[x]:
                vals = []
                info = (df.loc[i, 'RegionName'], df.loc[i, 'Cost'], df.loc[i, 'Return'])
                for x in range(1, len(info)):
                    newval = info[x]//increment
                    newval2 = (newval+1)*increment
                    vals.append(newval2)
                array = [info[0], vals[0], vals[1]]
                investmentsarray.append(array)
    invests = pd.DataFrame(investmentsarray)
    invests.columns = ['Region', 'Cost', 'Profit']
    profit2 = invests['Profit'].sum()


    print(invests)
    print("With Total Profit of:", '${:,.2f}'.format(profit2))

    
    newval = profit//increment
    profit2 = (newval+1)*increment
       
df = loadInvestments('Metro.csv')
profit, investments = optimizeInvestments(df, 1000000)

getinvests(df, investments, 1000)